In [46]:
import requests
import pandas as pd
#import nfl_data_py as nfl

def sleeper_get(url):
    """Basic GET wrapper."""
    r = requests.get(url)
    r.raise_for_status()
    return r.json()


def get_sleeper_players():
    """Returns a dict: player_id -> info (name, position, team, etc.)"""
    url = "https://api.sleeper.app/v1/players/nfl"
    return sleeper_get(url)


def get_league_rosters(league_id):
    """Returns list of rosters with player IDs."""
    url = f"https://api.sleeper.app/v1/league/{league_id}/rosters"
    return sleeper_get(url)


def get_weekly_projections(year, week):
    """Returns list of projections for the given week."""
    url = f"https://api.sleeper.app/v1/projections/nfl/{year}/{week}"
    return sleeper_get(url)


In [52]:
def build_lineup_df(league_id, owner_id=None):
    players = fetch_players()
    rosters = fetch_league_rosters(league_id)

    rows = []

    for roster in rosters:
        roster_owner = roster.get("owner_id")

        # Optional filter: skip other owners
        if owner_id is not None and roster_owner != owner_id:
            continue

        for pid in roster.get("players", []):
            info = players.get(pid)
            if info is None:
                continue

            name = f"{info.get('first_name','')} {info.get('last_name','')}".strip()
            pos = info.get("position")

            if pos is None:
                continue

            rows.append({
                "player": name,
                "position": pos,
                "proj": 0.0,          # placeholder
                "owner_id": roster_owner
            })

    return pd.DataFrame(rows)


def move_flex_to_bottom(positions):
    """
    Move all FLEX and SUPERFLEX entries to the end.
    Group all FLEX first, then all SUPERFLEX.
    """
    norm = [str(p).upper() for p in positions]

    non_flex = [n for n in norm if n not in ("FLEX", "SUPER_FLEX")]
    flex = [n for n in norm if n == "FLEX"]
    sflex = [n for n in norm if n == "SUPER_FLEX"]

    return non_flex + flex + sflex


def optimize_lineup(required, roster):
    # Step 1: reorder
    required = move_flex_to_bottom(required)

    # Step 2: output lineup
    lineup = []

    for need in required:

        if need == "FLEX":
            # FLEX can take RB/WR/TE
            eligible_positions = {"RB", "WR", "TE"}

        elif need == "SUPER_FLEX":
            # SUPERFLEX can take QB/RB/WR/TE
            eligible_positions = {"QB", "RB", "WR", "TE"}

        else:
            # Normal position: must match exactly
            eligible_positions = {need}

        # Filter roster for eligible players
        eligible_players = roster[roster['position'].isin(eligible_positions)]
        
        #Avoid null append error
        if eligible_players.empty:
            lineup.append(None)
            continue


        # Pick highest-projection row
        best = eligible_players.loc[eligible_players['proj'].idxmax()]
        lineup.append(best)

        # Remove from roster
        roster = roster[roster['player'] != best['player']]


    return lineup

In [54]:
league_id = "1262886410906509312" #Big Bongs
owner_id = '1264468857251188736' #Marc

roster_df = build_lineup_df(league_id,owner_id)
roster_df['proj'] = range(0,len(roster_df)) #Dummy Projs

required_positions = ['QB', 'RB', 'RB', 'WR', 'WR','TE', 'FLEX', 'FLEX','SUPER_FLEX','K', 'DEF']
lineup = optimize_lineup(required_positions, roster_df)

print(roster_df)
for p in lineup:
    print(p)

                    player position  proj             owner_id
0          Chris Rodriguez       RB     0  1264468857251188736
1           Brandon Aubrey        K     1  1264468857251188736
2           Caleb Williams       QB     2  1264468857251188736
3              Rome Odunze       WR     3  1264468857251188736
4            Ladd McConkey       WR     4  1264468857251188736
5             Tyrone Tracy       RB     5  1264468857251188736
6   Jacory Croskey-Merritt       RB     6  1264468857251188736
7          Jacoby Brissett       QB     7  1264468857251188736
8             Alvin Kamara       RB     8  1264468857251188736
9           Tua Tagovailoa       QB     9  1264468857251188736
10              Joe Burrow       QB    10  1264468857251188736
11       Amon-Ra St. Brown       WR    11  1264468857251188736
12       Wan'Dale Robinson       WR    12  1264468857251188736
13            Trey McBride       TE    13  1264468857251188736
14           Khalil Shakir       WR    14  126446885725